In [ ]:
# set up and test simple commands with LLM 
# swaped OpenAI for Mistral due to expired free tier quota

In [2]:
# import mistral 
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [1]:
import os

In [3]:
# get API key stored in env variable
api_key = os.getenv("MISTRAL_KEY")

In [4]:
client = MistralClient(api_key=api_key)

In [5]:
# ask it a question
response = client.chat(
    model = 'mistral-tiny',
    messages = [ChatMessage(role = "user", content = "Is it too late to join the course?")]
)

In [6]:
print(response.choices[0].message.content)

I'm sorry for any confusion, but I'll need a bit more information to accurately answer your question. The phrase "the course" is quite general. Are you asking about a specific course at a school, an online learning platform, or perhaps a workshop or seminar? Usually, the answer to "Is it too late to join the course?" depends on the course's start and end dates, as well as the policy of the organization offering the course. If you have more details, please provide them so I can give a more precise answer.


In [7]:
# set up a search using minisearch from Alexei Grigorev
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-30 19:35:37--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-30 19:35:37 (8.97 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [5]:
import minsearch

In [6]:
import json

In [10]:
# get the document
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-30 19:35:43--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.1’

documents.json.1    100%[===================>] 642.90K  --.-KB/s    in 0.02s   

2024-06-30 19:35:43 (26.7 MB/s) - ‘documents.json.1’ saved [658332/658332]



In [7]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [8]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [10]:
# index (fit the index to) the documents
index.fit(documents)

In [7]:
# some query (question) 
query = 'the course has already started, can I still enroll?'

In [8]:
# let's search but make some fields more important and some less so
boost = {'question': 3.0, 'section': 0.5}
results = index.search(
    query,
    boost_dict = boost, 
    # filter_dict = {'course': 'data-engineering-zoomcamp'}, # if we want to search only for specific course 
    num_results = 5
)

In [9]:
print(results)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}, {'text': 'Yes, we will keep all the materials after the course finishes, s

In [ ]:
# combine LLM and search

In [18]:
# create the prompt template
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [19]:
# define the context
context = ""
for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [21]:
prompt = prompt_template.format(question = query, context = context).strip()

In [24]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fi

In [11]:
# feed it to LLM
response = client.chat(
    model = "mistral-large-latest",
    messages = [ChatMessage(role = "user", content = prompt)]
)
print(response.choices[0].message.content)

NameError: name 'prompt' is not defined

In [ ]:
# clean up the flow

In [12]:
# define a function for search
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    search_results = index.search(
        query,
        boost_dict = boost, 
        # filter_dict = {'course': 'data-engineering-zoomcamp'}, # if we want to ask only about one specific course 
        num_results = 10
    )
    return search_results
    

In [13]:
# define a function for building a prompt
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE

    QUESTION: {question}

    CONTEXT: {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question = query, context = context).strip()
    return prompt
    

In [14]:
# define a function to talk to LLM
def llm(prompt):
    response = client.chat(
    model = "mistral-large-latest",
    messages = [ChatMessage(role = "user", content = prompt)]
    )    
    return response.choices[0].message.content

In [17]:
# call these functions within in one other function
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [18]:
print(rag('the course has already started, can I still enroll?'))

Yes, you can still enroll in the course even if it has already started. You won't be able to submit some of the homeworks, but you can still take part in the course. In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course late and manage to work on two projects, you will still be eligible for a certificate.


In [ ]:
# replace Alexey's search with elasticsearch

In [ ]:
# TODO: Elasticsearch, HW